# A example for how to crawler the website data from douban Top 250 movies

@author: CRQ

@affication: DUFE

@email: riqingchen@qq.com

@version: 0.1

In [245]:
# import the modules
import os
import re
import requests
import pandas as pd
from bs4 import BeautifulSoup
import random
import time
# import json

In [246]:
# define the my http header
myHttpheader = {  
  "User-Agent": r"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.99 Safari/537.36",
  "Accept": r"text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
  "Accept-Language": r"zh-CN,zh;q=0.8",
  "Connection": r"keep-alive",
  "Accept-Encoding": r"gzip, deflate, sdch"  
}

Define the function to get all top 250 movies urls.

In [247]:
def get_top250_urls(url):
    # url = starts[0]
    try: 
        web = requests.get(url, headers = myHttpheader)
        web.encoding = 'utf-8' # important
        soup = BeautifulSoup(web.text, 'lxml')
        pattern = r'https://movie.douban.com/subject/'
        urls = [x['href'] for x in soup.findAll('a', href = re.compile(pattern, re.I))]
        urls = list(set(urls))
    except:
        print(web.status_code)
        pirnt('%s is not found' %(url))
        urls = []
    return urls

Define the function to get the information of a movie.

In [248]:
# info_all_name = [x.text for x in soup.find_all('span', class_ = 'pl') if x.find_parents('div', id = 'info') != []]
def get_movie_info(url):
    info = {}
    try:
        web = requests.get(url, headers = myHttpheader)
        web.encoding = 'utf-8' # important
        soup = BeautifulSoup(web.text, 'lxml') 
        title_year = re.split(re.compile('[\(\)]'),re.sub('\n','',soup.findAll('h1')[0].text))
        info['movie_name'] = title_year[0]
        info['year'] = title_year[1]
        info_all = soup.findAll('div', id = "info")[0].text
        info_all = [re.split("[:：]",x.strip(), maxsplit=1) for x in re.split('\n', info_all) if x != '']
        info.update(dict(info_all))
        info['rate'] = soup.find(class_="ll rating_num").text
        info['people'] = soup.find('span',property="v:votes").text
        rate_star = dict(zip(['5星','4星','3星','2星','1星'],[x.text for x in soup.find_all('span',class_="rating_per")]))
        info.update(rate_star)        
    except:
        print(web.status_code)
        print('%s is not found' %(url))

    return info

In [249]:
if __name__ == '__main__':
    # set the work file directory
    path = r'J:\Rcrawler'
    if not os.path.exists(path):
        os.mkdir(path)
    os.chdir(path)
    
    # Get the start urls.
    start_urls = [r'https://movie.douban.com/top250?start=' + str(x * 25) + r'&filter=' for x in range(10) ]
    
    top250_urls = []
    for url in start_urls:
        top250_urls = top250_urls + get_top250_urls(url)
    info_all = []    
    for url in top250_urls:
        time.sleep(random.randint(1,5)/10)
        info = get_movie_info(url)
        info_all = info_all + [info]
        
    info_all = pd.DataFrame(info_all)
    

200
https://movie.douban.com/subject/1889243/ is not found
200
https://movie.douban.com/subject/3541415/ is not found
200
https://movie.douban.com/subject/1929463/ is not found
200
https://movie.douban.com/subject/3742360/ is not found
200
https://movie.douban.com/subject/3011235/ is not found
200
https://movie.douban.com/subject/3443389/ is not found
200
https://movie.douban.com/subject/1291543/ is not found


In [250]:
info_all

,1星,2星,3星,4星,5星,IMDb链接,movie_name,people,rate,year,上映日期,主演,制片国家/地区,又名,官方网站,导演,片长,类型,编剧,语言
0,0.1%,0.1%,1.5%,13.9%,84.4%,tt0111161,肖申克的救赎 The Shawshank Redemption,1447076,9.6,1994,1994-09-10(多伦多电影节) / 1994-10-14(美国),蒂姆·罗宾斯 / 摩根·弗里曼 / 鲍勃·冈顿 / 威廉姆·赛德勒 / 克兰西·布朗 / ...,美国,月黑高飞(港) / 刺激1995(台) / 地狱诺言 / 铁窗岁月 / 消香克的救赎,NaN,弗兰克·德拉邦特,142分钟,剧情 / 犯罪,弗兰克·德拉邦特 / 斯蒂芬·金,英语
1,0.1%,0.2%,4.1%,25.2%,70.4%,tt0245429,千与千寻 千と千尋の神隠し,1065250,9.3,2001,2019-06-21(中国大陆) / 2001-07-20(日本),柊瑠美 / 入野自由 / 夏木真理 / 菅原文太 / 中村彰男 / 玉井夕海 / 神木隆之...,日本,神隐少女(台) / 千与千寻的神隐 / Spirited Away / A Voyage ...,http://toho.co.jp/sentochihiro/index2.html,宫崎骏,125分钟,剧情 / 动画 / 奇幻,宫崎骏,日语
2,0.2%,0.5%,5.3%,24.4%,69.6%,tt0068646,教父 The Godfather,514625,9.3,1972,1972-03-15(纽约首映) / 1972-03-24(美国),马龙·白兰度 / 阿尔·帕西诺 / 詹姆斯·肯恩 / 理查德·卡斯特尔诺 / 罗伯特·杜瓦...,美国,Mario Puzo's The Godfather,NaN,弗朗西斯·福特·科波拉,175 分钟,剧情 / 犯罪,马里奥·普佐 / 弗朗西斯·福特·科波拉,英语 / 意大利语 / 拉丁语
3,NaN,NaN,NaN,NaN,NaN,NaN,星际穿越 Interstellar,NaN,NaN,2014,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.1%,0.3%,4.7%,23.4%,71.5%,tt1028532,忠犬八公的故事 Hachi: A Dog's Tale,755075,9.3,2009,2009-06-13(西雅图电影节) / 2010-03-12(英国),理查·基尔 / 萨拉·罗默尔 / 琼·艾伦 / 罗比·萨布莱特 / 艾瑞克·阿瓦利 / 田...,美国 / 英国,忠犬小八(台) / 秋田犬八千(港) / 忠犬传奇,NaN,拉斯·霍尔斯道姆,93分钟,剧情,斯蒂芬·P·林赛 / 新藤兼人,英语 / 日语
5,0.1%,0.2%,4.3%,24.3%,71.1%,tt0910970,机器人总动员 WALL·E,754298,9.3,2008,2008-06-27(美国),本·贝尔特 / 艾丽莎·奈特 / 杰夫·格尔林 / 佛莱德·威拉特 / 西格妮·韦弗 / ...,美国,瓦力(台) / 太空奇兵·威E(港) / 星际总动员 / 地球废品分装员 / 机器人瓦力 ...,www.pixar.com/feature-films/walle,安德鲁·斯坦顿,98分钟,爱情 / 科幻 / 动画 / 冒险,安德鲁·斯坦顿 / 吉姆·里尔顿 / 彼特·道格特,英语
6,0.1%,0.2%,2.2%,16.0%,81.4%,tt0106332,霸王别姬,1072072,9.6,1993,1993-01-01(香港),张国荣 / 张丰毅 / 巩俐 / 葛优 / 英达 / 蒋雯丽 / 吴大维 / 吕齐 / 雷...,中国大陆 / 香港,再见，我的妾 / Farewell My Concubine,NaN,陈凯歌,171 分钟,剧情 / 爱情 / 同性,芦苇 / 李碧华,汉语普通话
7,0.1%,0.1%,2.6%,19.8%,77.4%,tt0108052,辛德勒的名单 Schindler's List,594011,9.5,1993,1993-11-30(华盛顿首映) / 1994-02-04(美国),连姆·尼森 / 本·金斯利 / 拉尔夫·费因斯 / 卡罗琳·古多尔 / 乔纳森·萨加尔 /...,美国,舒特拉的名单(港) / 辛德勒名单,NaN,史蒂文·斯皮尔伯格,195分钟,剧情 / 历史 / 战争,托马斯·肯尼利 / 斯蒂文·泽里安,英语 / 希伯来语 / 德语 / 波兰语
8,0.1%,0.2%,2.4%,17.7%,79.6%,tt0118799,美丽人生 La vita è bella,667207,9.5,1997,1997-12-20(意大利),罗伯托·贝尼尼 / 尼可莱塔·布拉斯基 / 乔治·坎塔里尼 / 朱斯蒂诺·杜拉诺 / 赛尔...,意大利,一个快乐的传说(港) / Life Is Beautiful,NaN,罗伯托·贝尼尼,116分钟 / 125分钟(加长版),剧情 / 喜剧 / 爱情 / 战争,温琴佐·切拉米 / 罗伯托·贝尼尼,意大利语 / 德语 / 英语
9,0.3%,0.5%,6.1%,26.0%,67.2%,tt0114996,大话西游之大圣娶亲 西遊記大結局之仙履奇緣,794619,9.2,1995,2014-10-24(中国大陆) / 2017-04-13(中国大陆重映) / 1995-...,周星驰 / 吴孟达 / 朱茵 / 蔡少芬 / 蓝洁瑛 / 莫文蔚 / 罗家英 / 刘镇伟 ...,香港 / 中国大陆,西游记完结篇仙履奇缘 / 齐天大圣西游记 / 大话东游之二 / A Chinese Ody...,NaN,刘镇伟,95分钟 / 110分钟(重映版),喜剧 / 爱情 / 奇幻 / 古装,刘镇伟 / 吴承恩,粤语 / 汉语普通话
